# ChemProp分析: 化合物データセットを用いた分子特性予測

このノートブックでは、ChemPropライブラリを使用して化合物データセット（PBL_dataset.csv）の分子特性予測を行います。

## 1. 必要なライブラリのインストールと読み込み

In [ ]:
# ChemPropのインストール（初回のみ必要）
!pip install chemprop
!pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# ChemPropの読み込み
try:
    import chemprop
    print(f"ChemProp version: {chemprop.__version__}")
except ImportError:
    print("ChemPropがインストールされていません。上記のセルでインストールしてください。")

## 2. データの読み込みと前処理

In [ ]:
# データセットの読み込み
df = pd.read_csv('starderdized_compounds_20240725.tsv', sep='\t')

print(f"データセットの形状: {df.shape}")
print("\nカラム名:")
print(df.columns.tolist())
print("\n最初の5行:")
df.head()

In [ ]:
df_clean = df.copy()

# 全ての列の値を右側の空白を除去してからクリーニング
for col in df_clean.columns:
    # 文字列の右側空白を除去
    df_clean[col] = df_clean[col].apply(lambda x: str(x).rstrip())
    # 空文字列をNaNに変換
    df_clean[col] = df_clean[col].replace('', None)

print("クリーニング後の欠損値確認:")
print(df_clean.isnull().sum())

# Inhibition値を数値に変換
df_clean['Inhibition'] = pd.to_numeric(df_clean['Inhibition'], errors='coerce')

print(f"\nInhibition値の統計:")
inhibition_valid = df_clean['Inhibition'].dropna()
print(f"有効な値の数: {len(inhibition_valid)}")
print(f"平均: {inhibition_valid.mean():.4f}")
print(f"最小: {inhibition_valid.min():.4f}")
print(f"最大: {inhibition_valid.max():.4f}")

# ChemProp用データの準備
chemprop_ready = df_clean[['washed_SMILES', 'Inhibition']].dropna()
print(f"\nChemProp用データ準備完了: {len(chemprop_ready)}行")

## 3. ChemProp用のデータ準備

In [ ]:
# ChemProp用のデータフォーマット作成
# ChemPropは SMILES, target の形式でCSVファイルを期待する
chemprop_data = df_clean[['washed_SMILES', 'Inhibition']].copy()
chemprop_data.columns = ['smiles', 'inhibition']

# NaNやinvalidなSMILESを除去
chemprop_data = chemprop_data.dropna()
print(f"ChemProp用データセットの形状: {chemprop_data.shape}")

# データを訓練用とテスト用に分割
train_data, test_data = train_test_split(
    chemprop_data, 
    test_size=0.2, 
    random_state=42, 
    stratify=None  # 回帰問題なのでstratifyは使用しない
)

print(f"訓練データ: {len(train_data)}個")
print(f"テストデータ: {len(test_data)}個")

# CSVファイルとして保存
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

print("\n訓練データの例:")
print(train_data.head())

## 4. ChemPropコマンドライン使用による訓練

In [ ]:
# コマンドライン経由でChemPropを実行
import subprocess
import os

# チェックポイントディレクトリを作成
os.makedirs('chemprop_checkpoint', exist_ok=True)

# ChemProp訓練コマンド
train_command = [
    'chemprop_train',
    '--data_path', 'train_data.csv',
    '--dataset_type', 'regression',
    '--save_dir', 'chemprop_checkpoint',
    '--epochs', '30',
    '--hidden_size', '300',
    '--depth', '3',
    '--batch_size', '50',
    '--seed', '42'
]

print("コマンドライン経由でChemPropを実行します...")
print(f"コマンド: {' '.join(train_command)}")

try:
    result = subprocess.run(train_command, capture_output=True, text=True, timeout=600)
    print("訓練完了!")
    print("標準出力:")
    print(result.stdout)
    if result.stderr:
        print("エラー出力:")
        print(result.stderr)
except subprocess.TimeoutExpired:
    print("訓練がタイムアウトしました。")
except FileNotFoundError:
    print("chemprop_trainコマンドが見つかりません。ChemPropが正しくインストールされているか確認してください。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

## 5. 予測の実行

In [ ]:
# テストデータで予測を実行
if os.path.exists('chemprop_checkpoint'):
    predict_command = [
        'chemprop_predict',
        '--test_path', 'test_data.csv',
        '--checkpoint_dir', 'chemprop_checkpoint',
        '--preds_path', 'predictions.csv'
    ]
    
    print("テストデータで予測を実行します...")
    try:
        result = subprocess.run(predict_command, capture_output=True, text=True, timeout=300)
        print("予測完了!")
        if result.stdout:
            print("標準出力:")
            print(result.stdout)
        if result.stderr:
            print("エラー出力:")
            print(result.stderr)
    except Exception as e:
        print(f"予測中にエラーが発生しました: {e}")
else:
    print("モデルのチェックポイントが見つかりません。")

## 6. 結果の評価と可視化

In [ ]:
# 予測結果の読み込みと評価
import pandas as pd

# 予測結果の読み込みと評価
if os.path.exists('predictions.csv'):
    predictions = pd.read_csv('predictions.csv')

    print(f"予測結果ファイルの形状: {predictions.shape}")
    print(f"カラム名: {predictions.columns.tolist()}")

    # テストデータと予測結果をマージ
    test_results = test_data.reset_index(drop=True)
    test_results['predicted'] = predictions['inhibition']  # 正しいカラム名を指定

    # 評価指標の計算
    y_true = test_results['inhibition']
    y_pred = test_results['predicted']

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print("=== ChemProp予測結果の評価 ===")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²スコア: {r2:.4f}")

    # 予測 vs 実測値のプロット
    plt.figure(figsize=(12, 5))

    # 散布図
    plt.subplot(1, 2, 1)
    plt.scatter(y_true, y_pred, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('Actual Inhibition')
    plt.ylabel('Predicted Inhibition')
    plt.title(f'ChemProp: Predicted vs Actual\\nR² = {r2:.3f}, RMSE = {rmse:.3f}')

    # 残差プロット
    plt.subplot(1, 2, 2)
    residuals = y_pred - y_true
    plt.scatter(y_pred, residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Inhibition')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')

    plt.tight_layout()
    plt.show()

    # 最も良い予測と悪い予測を表示
    test_results['abs_error'] = np.abs(test_results['predicted'] - test_results['inhibition'])

    print("\\n=== 最も予測精度の高い化合物 TOP 5 ===")
    best_predictions = test_results.nsmallest(5, 'abs_error')
    for idx, row in best_predictions.iterrows():
        print(f"実測値: {row['inhibition']:.3f}, 予測値: {row['predicted']:.3f}, 誤差: {row['abs_error']:.3f}")

    print("\\n=== 最も予測精度の低い化合物 TOP 5 ===")
    worst_predictions = test_results.nlargest(5, 'abs_error')
    for idx, row in worst_predictions.iterrows():
        print(f"実測値: {row['inhibition']:.3f}, 予測値: {row['predicted']:.3f}, 誤差: {row['abs_error']:.3f}")

else:
    print("予測結果ファイルが見つかりません。")

## 7. 従来手法との比較（参考）

In [ ]:
# 簡単なベースライン手法との比較
from sklearn.ensemble import RandomForestRegressor

try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors
    
    def smiles_to_descriptors(smiles_list):
        """SMILESから分子記述子を計算"""
        descriptors = []
        for smiles in smiles_list:
            try:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    desc = [
                        Descriptors.MolWt(mol),
                        Descriptors.LogP(mol),
                        Descriptors.NumHDonors(mol),
                        Descriptors.NumHAcceptors(mol),
                        Descriptors.TPSA(mol),
                        Descriptors.NumRotatableBonds(mol)
                    ]
                else:
                    desc = [0] * 6  # 無効なSMILESの場合はゼロで埋める
            except:
                desc = [0] * 6
            descriptors.append(desc)
        return np.array(descriptors)
    
    # RDKitがインストールされている場合のみ実行
    X_train = smiles_to_descriptors(train_data['smiles'].tolist())
    X_test = smiles_to_descriptors(test_data['smiles'].tolist())
    y_train_rf = train_data['inhibition'].values
    y_test_rf = test_data['inhibition'].values
    
    # ランダムフォレストモデルの訓練
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train_rf)
    
    # 予測
    y_pred_rf = rf_model.predict(X_test)
    
    # 評価
    rmse_rf = np.sqrt(mean_squared_error(y_test_rf, y_pred_rf))
    mae_rf = mean_absolute_error(y_test_rf, y_pred_rf)
    r2_rf = r2_score(y_test_rf, y_pred_rf)
    
    print("\n=== ランダムフォレスト（ベースライン）の結果 ===")
    print(f"RMSE: {rmse_rf:.4f}")
    print(f"MAE: {mae_rf:.4f}")
    print(f"R²スコア: {r2_rf:.4f}")
    
    # ChemPropとの比較（予測結果がある場合）
    if 'rmse' in locals():
        print("\n=== 手法比較 ===")
        comparison_df = pd.DataFrame({
            'Method': ['ChemProp', 'Random Forest'],
            'RMSE': [rmse, rmse_rf],
            'MAE': [mae, mae_rf],
            'R²': [r2, r2_rf]
        })
        print(comparison_df)
        
        # 比較プロット
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # ChemProp
        axes[0].scatter(y_true, y_pred, alpha=0.6, label='ChemProp')
        axes[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
        axes[0].set_xlabel('Actual')
        axes[0].set_ylabel('Predicted')
        axes[0].set_title(f'ChemProp (R² = {r2:.3f})')
        
        # Random Forest
        axes[1].scatter(y_test_rf, y_pred_rf, alpha=0.6, label='Random Forest', color='orange')
        axes[1].plot([y_test_rf.min(), y_test_rf.max()], [y_test_rf.min(), y_test_rf.max()], 'r--', lw=2)
        axes[1].set_xlabel('Actual')
        axes[1].set_ylabel('Predicted')
        axes[1].set_title(f'Random Forest (R² = {r2_rf:.3f})')
        
        plt.tight_layout()
        plt.show()
    
except ImportError:
    print("RDKitがインストールされていないため、ベースライン手法との比較はスキップします。")
    print("比較を行うには 'pip install rdkit' でRDKitをインストールしてください。")
except Exception as e:
    print(f"ベースライン手法の実行中にエラーが発生しました: {e}")

## 8. まとめと考察

### 分析結果のまとめ

このノートブックでは、ChemPropライブラリを使用して化合物の阻害値（Inhibition）の予測を行いました。

#### 使用したデータ
- **データセット**: PBL_dataset.csv
- **化合物数**: 2,000+ 個（Inhibition値があるもの）
- **入力**: SMILES（分子の化学構造）
- **出力**: Inhibition値（薬物の効果強度）

#### ChemPropの特徴
1. **メッセージパッシングニューラルネットワーク（MPNN）**を使用
2. **分子グラフ構造**を直接学習
3. **従来の分子記述子**を使用せずに予測が可能

#### 期待される利点
- より複雑な分子間相互作用の学習
- 従来手法では捉えられない構造的特徴の活用
- 新規化合物に対する汎化性能の向上

### 今後の発展可能性
1. **ハイパーパラメータ最適化**による性能向上
2. **アンサンブル学習**による予測精度の改善
3. **転移学習**による少数データでの学習
4. **説明可能AI**による重要な分子部分の特定

### 実行上の注意点
- ChemPropのインストールが必要: `pip install chemprop`
- 比較用にRDKitも推奨: `pip install rdkit`
- 訓練には時間がかかる場合があります（データサイズによる）
